# Notebook to demonstrate multiscale cleaning on the canonical "M31" source.

We simulate some VLA data for M31 and then use msclean to restore it.

In [ ]:
%matplotlib inline

import sys, os 
print(os.path.abspath('.'))
sys.path.append(os.path.join('..','..')) 

import pylab
pylab.rcParams['figure.figsize'] = (12.0, 12.0)
pylab.rcParams['image.cmap'] = 'rainbow'

import numpy
import scipy
import scipy.special

from matplotlib import pylab
from matplotlib import pyplot as plt

import pyfits

from crocodile.synthesis import *
from crocodile.simulate import *
from util.visualize import *
from msclean import *

Now generate and plot the uvw coordinates

In [ ]:
vlas=numpy.genfromtxt("../../data/vis/VLA_A_hor_xyz.txt", delimiter=",") 
vlas_xyz = xyz_at_latitude(vlas, numpy.pi*34.0784/180.0)/30.0 
uvw=xyz_to_baselines(vlas_xyz, 
numpy.arange(-numpy.pi/2.0, +numpy.pi/2.0,0.1), 
numpy.pi/8)
plt.clf()
plt.plot(uvw[:,0],uvw[:,1], '.')
plt.plot(-uvw[:,0],-uvw[:,1], '.', color='red')
plt.xlabel('U (m)')
plt.ylabel('V (m)')
plt.title('UV coverage')

Next get the M31 model from the crocodile data directory

In [ ]:
m31model='../../data/models/M31.MOD'
hdulist = pyfits.open(m31model)
m31image=numpy.array(hdulist[0].data)
plt.clf()
plt.imshow(m31image, cmap='rainbow', origin='lower')
plt.show()

Calculate the visibility for m31 using wslicfwd

In [ ]:
lam=4.0*numpy.abs(uvw).max()
cell=1.0/lam
theta=256*cell

print("Field of view = %.2f degrees, maximum baseline %.3f (m)" % (theta*180.0/numpy.pi, lam))
pointimage=m31image.copy()
pointimage[225,75]+=2.0
pointimage[25,100]+=4.0
uvwsort,vobs=dopredict(theta, lam, p=uvw, modelimage=m31image, predfn=wslicfwd)

Plot the visibility as a function of uv distance

In [ ]:
uvdist=numpy.sqrt(uvwsort[:,0]*uvwsort[:,0]+uvwsort[:,1]*uvwsort[:,1])
plt.clf()
plt.plot(uvdist, numpy.abs(vobs), '.')
plt.xlabel('UV distance (wavelengths)')
plt.ylabel('Amp Visibility')
plt.show()

Next we make the dirty image and psf to check if it looks plausible

In [ ]:
d,p=doimg(2.0*theta, lam, uvwsort, vobs, wslicimg)
plt.clf()
plt.subplot(1,2,1)
plt.imshow(p, cmap='rainbow', origin='lower')
plt.title('PSF')
c=plt.colorbar()
plt.subplot(1,2,2)
plt.imshow(d, cmap='rainbow', origin='lower')
plt.title('Dirty image')
c=plt.colorbar()



Now we can do the deconvolution

In [ ]:
scales=[0, 3, 10, 30]
ps, vs, m31cc, m31res=majorcycle(2.0*theta, lam, uvwsort, vobs, 0.75, 5, 100, 200, scales, 0.001, 0.1)

Finally show the results

In [ ]:
print("Total flux in model %.6f Jy" % numpy.sum(m31cc))
plt.clf()
plt.subplot(1,2,1)
plt.imshow(m31cc, cmap='rainbow', origin='lower')
c=plt.colorbar()
plt.subplot(1,2,2)
plt.imshow(m31res, cmap='rainbow', origin='lower')
c=plt.colorbar()




In [ ]:
plt.clf()
plt.plot(uvdist, numpy.abs(vs), '.')
plt.xlabel('UV distance (wavelengths)')
plt.ylabel('Amp Residual Visibility')
plt.show()

In [ ]:
plt.clf()
plt.plot(numpy.abs(vs), '.')
plt.xlabel('Index')
plt.ylabel('Amp Residual Visibility')
plt.show()